# End-to-End Example

In [10]:

%pip install fsspec langchain-text-splitters openai python-dotenv numpy torch

  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 49.7 MB/s eta 0:00:00a 0:00:01
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 30.3 MB/s eta 0:00:00
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install -U git+https://github.com/neo4j/neo4j-graphrag-python

  Cloning https://github.com/neo4j/neo4j-graphrag-python to /private/var/folders/dq/tkr53kq51wd04bk26m5s3_5c0000gn/T/pip-req-build-6h32lpd3
  Running command git clone --filter=blob:none --quiet https://github.com/neo4j/neo4j-graphrag-python /private/var/folders/dq/tkr53kq51wd04bk26m5s3_5c0000gn/T/pip-req-build-6h32lpd3
  Resolved https://github.com/neo4j/neo4j-graphrag-python to commit ff1c6ee677996f804d8f80c669aae52a4e317925
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [12]:
from dotenv import load_dotenv
import os

#load neo4j credentials

load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [13]:
from neo4j_graphrag.experimental.components.pdf_loader import PdfLoader
loader = PdfLoader()
pdf_doc = await loader.run("pgpm-13-39.pdf")

In [14]:
from langchain_text_splitters import CharacterTextSplitter
from neo4j_graphrag.experimental.components.text_splitters.langchain import LangChainTextSplitterAdapter
splitter = LangChainTextSplitterAdapter(
    CharacterTextSplitter(chunk_size=500, chunk_overlap=100, separator=".")
)
split_text = await splitter.run(text=pdf_doc.text)

Created a chunk of size 563, which is longer than the specified 500
Created a chunk of size 646, which is longer than the specified 500
Created a chunk of size 732, which is longer than the specified 500


In [15]:
split_text.chunks[0].text

'REVIEW\nT owards Precision Medicine in Systemic Lupus\nErythematosus\nThis article was published in the following Dove Press journal:\nPharmacogenomics and Personalized Medicine\nElliott Lever1\nMarta R Alves2\nDavid A Isenberg1\n1Centre for Rheumatology, Division of\nMedicine, University College Hospital\nLondon, London, UK;2Internal Medicine,\nDepartment of Medicine, Centro\nHospitalar do Porto, Porto, PortugalAbstract: Systemic lupus erythematosus (SLE) is a remarkable condition characterised by\ndiversity amongst its clinical features and immunological abnormalities'

In [16]:
from neo4j_graphrag.experimental.components.schema import (
    SchemaBuilder,
    SchemaEntity,
    SchemaProperty,
    SchemaRelation,
)

schema_builder = SchemaBuilder()

node_properties = [SchemaProperty(name="name", type="STRING"), SchemaProperty(name="details", type="STRING")]

basic_node_labels = ["Object",
                     "Entity",
                     "Group",
                     "Person",
                     "Organization",
                     "Place"
                     ]
academic_node_labels = ["ArticleOrPaper", "PublicationOrJournal"]

medical_node_labels = ["Anatomy",
                       "BiologicalProcess",
                       "Cell",
                       "CellularComponent",
                       "CellType",
                       "Condition",
                       "Disease",
                       "Drug",
                       "EffectOrPhenotype",
                       "Exposure",
                       "GeneOrProtein",
                       "Molecule",
                       "MolecularFunction",
                       "Pathway"
                       ]

node_labels = basic_node_labels + academic_node_labels + medical_node_labels


rel_properties = [SchemaProperty(name="relType", type="STRING", description="The type of relationship")]
basic_rel_types = ["RELATES_TO"]

# cartesian product for possible schema
possible_schema = []
for src in node_labels:
    for dst in node_labels:
        for re in basic_rel_types:
            possible_schema.append((src, re, dst))


In [17]:
schema = await schema_builder.run(
    entities=[SchemaEntity(label=i, properties=node_properties) for i in node_labels],
    relations=[] ,#[SchemaRelation(label=i, properties=rel_properties) for i in basic_rel_types],
    potential_schema=[] #possible_schema
)
schema

SchemaConfig(entities={'Object': {'label': 'Object', 'description': '', 'properties': [{'name': 'name', 'type': 'STRING', 'description': ''}, {'name': 'details', 'type': 'STRING', 'description': ''}]}, 'Entity': {'label': 'Entity', 'description': '', 'properties': [{'name': 'name', 'type': 'STRING', 'description': ''}, {'name': 'details', 'type': 'STRING', 'description': ''}]}, 'Group': {'label': 'Group', 'description': '', 'properties': [{'name': 'name', 'type': 'STRING', 'description': ''}, {'name': 'details', 'type': 'STRING', 'description': ''}]}, 'Person': {'label': 'Person', 'description': '', 'properties': [{'name': 'name', 'type': 'STRING', 'description': ''}, {'name': 'details', 'type': 'STRING', 'description': ''}]}, 'Organization': {'label': 'Organization', 'description': '', 'properties': [{'name': 'name', 'type': 'STRING', 'description': ''}, {'name': 'details', 'type': 'STRING', 'description': ''}]}, 'Place': {'label': 'Place', 'description': '', 'properties': [{'name': '

In [18]:
from neo4j_graphrag.experimental.components.entity_relation_extractor import (
    LLMEntityRelationExtractor, OnError,
)
from neo4j_graphrag.llm import OpenAILLM

extractor = LLMEntityRelationExtractor(
    llm=OpenAILLM(
        model_name="gpt-4o",
        model_params={
            "max_tokens": 1000,
            "response_format": {"type": "json_object"},
        }
    ),
    on_error=OnError.IGNORE,
)


In [19]:
res_graph = await extractor.run(chunks=split_text, schema=schema)

/Users/zachblumenfeld/opt/anaconda3/envs/neo4j-graphrag-python-example/lib/python3.11/site-packages/neo4j_graphrag/experimental/components/entity_relation_extractor.py:424: UserWarning: No document metadata provided, the document node won't be created in the lexical graph
  warnings.warn(


In [20]:
print(len(res_graph.nodes))
print(len(res_graph.relationships))

1368
1552


In [21]:
import neo4j
from neo4j_graphrag.experimental.components.kg_writer import Neo4jWriter
from neo4j_graphrag.experimental.components.types import Neo4jGraph

with neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    writer = Neo4jWriter(driver)
    await writer.run(res_graph)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 1, column: 87, offset: 86} for query: 'MERGE (n:__Entity__ {id: $id}) WITH n SET n:`ArticleOrPaper`, n += $properties WITH n CALL { WITH n WITH n WHERE $embeddings IS NOT NULL UNWIND keys($embeddings) as emb CALL db.create.setNodeVectorProperty(n, emb, $embeddings[emb]) } RETURN elementId(n)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 1, column: 93, o